In [1]:
from pathlib import Path

import cv2
import ipywidgets
import pandas as pd
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

In [2]:
SZ            = 224                   # tamaño de las imágenes a etiquetar
IN_P          = Path('train_dataset') # directorio de fotos a etiquetar
HELPER_LINE_Y = 0.33                  # posición Y de la línea de ayuda (de 0 a 1)

In [3]:
files_l = list(IN_P.glob('*.jpg'))

In [4]:
label_widget = ipywidgets.Label()
style = {'description_width': 'initial'}
idx_widget = ipywidgets.BoundedIntText(
    description=f'Index [0 .. {len(files_l)-1}]', 
    min=0, 
    max=len(files_l)-1, 
    style=style)
prev_widget = ipywidgets.Button(description='<-')
next_widget = ipywidgets.Button(description='->')
save_widget = ipywidgets.Button(description='Save .csv')
image_widget = ClickableImageWidget(width=SZ, height=SZ)
snapshot_widget = ipywidgets.Image(width=SZ, height=SZ)

In [5]:
def load_image():
    global image
    idx = idx_widget.value
    image = cv2.imread(str(files_l[idx]), cv2.IMREAD_COLOR)
    cv2.line(image, (0, int(SZ * HELPER_LINE_Y)), (SZ, int(SZ * HELPER_LINE_Y)), (255, 255, 0), 1)
    if files_l[idx].name in data:
        x = data[files_l[idx].name]['x']
        y = data[files_l[idx].name]['y']
        cv2.circle(image, (x, y), 8, (0, 255, 255), 2)
    image_widget.value = bgr8_to_jpeg(image)
    label_widget.value = str(files_l[idx])

def on_click_next(obj):
    idx_widget.value += 1

def on_click_prev(obj):
    idx_widget.value -= 1
        
def on_idx_change(change):
    load_image()
    
def save_snapshot(_, content, msg):
    idx = idx_widget.value
    if content['event'] == 'click':
        e = content['eventData']
        x = e['offsetX']
        y = e['offsetY']
        data[files_l[idx].name] = { 'x': x, 'y': y }
        
        # display saved snapshot
        snapshot = cv2.circle(image, (x, y), 8, (0, 255, 0), 2)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        
        idx_widget.value += 1

def on_click_save(obj):
    pd.DataFrame.from_dict(data, 'index').rename_axis('filename').to_csv(IN_P / 'labels.csv')
        
image_widget.on_msg(save_snapshot)
prev_widget.on_click(on_click_prev)
next_widget.on_click(on_click_next)
save_widget.on_click(on_click_save)
idx_widget.observe(on_idx_change, names='value')


In [6]:
if (IN_P / 'labels.csv').exists():
    data = pd.read_csv(IN_P / 'labels.csv').set_index('filename').to_dict('index')
else:
    data = {}

In [7]:
idx_widget.value = 0
load_image()

In [8]:
data_collection_widget = ipywidgets.VBox([
    label_widget,
    ipywidgets.HBox([image_widget, snapshot_widget]),
    idx_widget,
    ipywidgets.HBox([
        prev_widget,
        next_widget
    ], width='500px'),
    save_widget
], width='80%')

In [9]:
display(data_collection_widget)